In [0]:
%sql
USE CATALOG c_bikeshop

In [0]:
%sql

CREATE TABLE IF NOT EXISTS silver.dim_brand (
  brand_id INT,
  brand_name STRING,
  row_hash STRING,
  updated_at TIMESTAMP

);

WITH staged AS (
  SELECT 
    brand_id,
    brand_name,
    SHA2(CONCAT_WS(brand_id, brand_name), 256) AS row_hash
  FROM bronze.brands

)

MERGE INTO silver.dim_brand t
USING staged s
ON t.brand_id = s.brand_id

WHEN MATCHED AND t.row_hash <> s.row_hash THEN
  UPDATE SET
    t.brand_name = s.brand_name,
    t.row_hash = s.row_hash,
    t.updated_at = current_timestamp() 

WHEN NOT MATCHED THEN
  INSERT (brand_id, brand_name, row_hash, updated_at)
  VALUES (s.brand_id, s.brand_name, s.row_hash, current_timestamp())

In [0]:
%sql
DELETE FROM silver.dim_brand WHERE brand_id NOT IN (SELECT brand_id FROM bronze.brands);

In [0]:
%sql

CREATE TABLE IF NOT EXISTS silver.dim_categories (
  category_id	int,
  category_name	string,
  row_hash string,
  updated_at timestamp
);

WITH staged AS (
  SELECT 
    category_id,
    category_name,
    SHA2(CONCAT_WS(category_id, category_name), 256) AS row_hash
  FROM bronze.categories
)

MERGE INTO silver.dim_categories t
USING staged s
ON t.category_id = s.category_id

WHEN MATCHED AND t.row_hash <> s.row_hash THEN
  UPDATE SET
    t.category_name = s.category_name,
    t.row_hash = s.row_hash,
    t.updated_at = current_timestamp() 

WHEN NOT MATCHED THEN
  INSERT (category_id, category_name, row_hash, updated_at)
  VALUES (s.category_id, s.category_name, s.row_hash, current_timestamp());
    


In [0]:
%sql
DELETE FROM silver.dim_categories WHERE category_id NOT IN (SELECT category_id FROM bronze.categories)

In [0]:
%sql

CREATE TABLE IF NOT EXISTS silver.fact_orders (
  order_id	int,
  customer_id	int,
  order_status	int,
  order_date	date,
  required_date	date,
  shipped_date	date,
  store_id	int,
  staff_id	int,
  row_hash string,
  updated_at timestamp
);

WITH staged AS (
  SELECT 
    order_id,
    customer_id,
    order_status,
    TRY_CAST(order_date AS DATE) AS order_date,
    TRY_CAST(required_date AS DATE) AS required_date,
    TRY_CAST(shipped_date AS DATE) AS shipped_date,
    store_id,
    staff_id,
    SHA2(CONCAT_WS('|', order_id, customer_id, order_status, order_date,TRY_CAST(order_date AS DATE),TRY_CAST(required_date AS DATE), TRY_CAST(shipped_date AS DATE), store_id, staff_id), 256) AS row_hash
  FROM bronze.orders
)

MERGE INTO silver.fact_orders t
USING staged s
ON t.order_id = s.order_id

WHEN MATCHED AND t.row_hash <> s.row_hash THEN
  UPDATE SET
    t.order_status = s.order_status,
    t.order_date = s.order_date,
    t.required_date = s.required_date,
    t.shipped_date = s.shipped_date,
    t.store_id = s.store_id,
    t.staff_id = s.staff_id,
    t.row_hash = s.row_hash,
    t.updated_at = current_timestamp() 

WHEN NOT MATCHED THEN 
  INSERT (
    order_id,
    customer_id,
    order_status,
    order_date,
    required_date,
    shipped_date,
    store_id,
    staff_id,
    row_hash,
    updated_at
  )
  VALUES(
    s.order_id,
    s.customer_id,
    s.order_status,
    s.order_date,
    s.required_date,
    s.shipped_date,
    s.store_id,
    s.staff_id,
    s.row_hash,
    current_timestamp()
  )

In [0]:
%sql
DELETE FROM silver.fact_orders WHERE order_id NOT IN (SELECT order_id FROM bronze.orders)

In [0]:

%sql


CREATE TABLE IF NOT EXISTS silver.dim_customer (
    customer_id INT,
    first_name  STRING,
    last_name   STRING,
    email       STRING,
    phone       STRING,
    street      STRING,
    city        STRING,
    state       STRING,
    zip_code    STRING,
    row_hash    STRING,
    updated_at  TIMESTAMP
);

WITH staged AS (
    SELECT
        *,
        SHA2(
            CONCAT_WS('|', customer_id, first_name, last_name,
                      email, phone, street, city, state, zip_code),
            256
        ) AS row_hash
    FROM bronze.customers
)

MERGE INTO silver.dim_customer t
USING staged s
ON t.customer_id = s.customer_id

WHEN MATCHED AND t.row_hash <> s.row_hash THEN
    UPDATE SET
        t.first_name = s.first_name,
        t.last_name  = s.last_name,
        t.email      = s.email,
        t.phone      = s.phone,
        t.street     = s.street,
        t.city       = s.city,
        t.state      = s.state,
        t.zip_code   = s.zip_code,
        t.row_hash   = s.row_hash,
        t.updated_at = current_timestamp()

WHEN NOT MATCHED THEN
    INSERT (
        customer_id, first_name, last_name, email, phone,
        street, city, state, zip_code, row_hash, updated_at
    )    
    VALUES (
        s.customer_id, s.first_name, s.last_name, s.email, s.phone,
        s.street, s.city, s.state, s.zip_code, s.row_hash, current_timestamp()
    );

In [0]:
%sql
DELETE FROM silver.dim_customer
WHERE customer_id NOT IN (SELECT customer_id FROM bronze.customers);

In [0]:
%sql

CREATE TABLE IF NOT EXISTS silver.dim_product (
    product_id   INT,
    product_name STRING,
    brand_id     INT,        
    category_id  INT,        
    model_year   INT,
    list_price   DOUBLE,
    row_hash     STRING,
    updated_at   TIMESTAMP
);

WITH staged AS (
    SELECT
        *,
        SHA2(
            CONCAT_WS('|', product_id, product_name, brand_id,
                      category_id, model_year, list_price),
            256
        ) AS row_hash
    FROM bronze.products
)

MERGE INTO silver.dim_product t
USING staged s
ON t.product_id = s.product_id

WHEN MATCHED AND t.row_hash <> s.row_hash THEN
    UPDATE SET
        t.product_name = s.product_name,
        t.brand_id     = s.brand_id,
        t.category_id  = s.category_id,
        t.model_year   = s.model_year,
        t.list_price   = s.list_price,
        t.row_hash     = s.row_hash,
        t.updated_at   = current_timestamp()

WHEN NOT MATCHED THEN
    INSERT (
        product_id, product_name, brand_id, category_id,
        model_year, list_price, row_hash, updated_at
    )
    VALUES (
        s.product_id, s.product_name, s.brand_id, s.category_id,
        s.model_year, s.list_price, s.row_hash, current_timestamp()
    );



In [0]:
%sql
DELETE FROM silver.dim_product
WHERE product_id NOT IN (SELECT product_id FROM bronze.products);

In [0]:
%sql


CREATE TABLE IF NOT EXISTS silver.dim_staff (
    staff_id   INT,
    first_name STRING,
    last_name  STRING,
    email      STRING,
    phone      STRING,
    active     INT,
    store_id   INT,
    manager_id INT,
    row_hash   STRING,
    updated_at TIMESTAMP
);

WITH staged AS (
    SELECT
        try_cast(staff_id as INT)   as staff_id,     
        first_name,
        last_name,
        email,
        phone,
        try_cast(active as INT)     as active,
        try_cast(store_id as INT)   as store_id,
        try_cast(manager_id as INT) as manager_id,
        SHA2(
            CONCAT_WS('|',
                try_cast(staff_id as INT),
                first_name,
                last_name,
                email,
                phone,
                try_cast(active as INT),
                try_cast(store_id as INT),
                try_cast(manager_id as INT)
            ),
            256
        ) AS row_hash
    FROM bronze.staffs
)

MERGE INTO silver.dim_staff t
USING staged s
ON t.staff_id = s.staff_id

WHEN MATCHED AND t.row_hash <> s.row_hash THEN
    UPDATE SET
        t.first_name = s.first_name,
        t.last_name  = s.last_name,
        t.email      = s.email,
        t.phone      = s.phone,
        t.active     = s.active,
        t.store_id   = s.store_id,
        t.manager_id = s.manager_id,
        t.row_hash   = s.row_hash,
        t.updated_at = current_timestamp()

WHEN NOT MATCHED THEN
    INSERT (
        staff_id, first_name, last_name, email, phone,
        active, store_id, manager_id, row_hash, updated_at
    )
    VALUES (
        s.staff_id, s.first_name, s.last_name, s.email, s.phone,
        s.active, s.store_id, s.manager_id, s.row_hash, current_timestamp()
    );



In [0]:
%sql
DELETE FROM silver.dim_staff
WHERE staff_id NOT IN (SELECT staff_id FROM bronze.staffs);

In [0]:
%sql

CREATE TABLE IF NOT EXISTS silver.dim_stores (
    store_id   INT,
    store_name STRING,
    phone      STRING,
    email      STRING,
    street     STRING,
    city       STRING,
    state      STRING,
    zip_code   STRING,
    row_hash   STRING,
    updated_at TIMESTAMP
);

WITH staged AS (
    SELECT
        *,
        SHA2(
            CONCAT_WS('|', store_id, store_name, phone, email,
                      street, city, state, zip_code),
            256
        ) AS row_hash
    FROM bronze.stores
)

MERGE INTO silver.dim_stores t
USING staged s
ON t.store_id = s.store_id

WHEN MATCHED AND t.row_hash <> s.row_hash THEN
    UPDATE SET
        t.store_name = s.store_name,
        t.phone      = s.phone,
        t.email      = s.email,
        t.street     = s.street,
        t.city       = s.city,
        t.state      = s.state,
        t.zip_code   = s.zip_code,
        t.row_hash   = s.row_hash,
        t.updated_at = current_timestamp()

WHEN NOT MATCHED THEN
    INSERT 
    (
        store_id, store_name, phone, email,
        street, city, state, zip_code,
        row_hash, updated_at
    )
    VALUES (
        s.store_id, s.store_name, s.phone, s.email,
        s.street, s.city, s.state, s.zip_code,
        s.row_hash, current_timestamp()
    );



In [0]:
%sql
DELETE FROM silver.dim_stores
WHERE store_id NOT IN (SELECT store_id FROM bronze.stores);

In [0]:
%sql

CREATE TABLE IF NOT EXISTS silver.fact_orders_items (
    order_id   INT,
    item_id    INT,
    product_id INT,        -- FK para dim_product
    quantity   INT,        -- Métrica: quantidade vendida
    list_price DOUBLE,     -- Métrica: preço de lista
    discount   DOUBLE,     -- Métrica: desconto aplicado
    row_hash   STRING,
    updated_at TIMESTAMP
);

WITH staged AS (
    SELECT
        *,
        SHA2(
            CONCAT_WS('|', order_id, item_id, product_id, quantity, list_price, discount),
            256
        ) AS row_hash
    FROM bronze.order_items
)

MERGE INTO silver.fact_orders_items t
USING staged s
ON t.order_id = s.order_id AND t.item_id = s.item_id   -- Chave composta na condição de junção

WHEN MATCHED AND t.row_hash <> s.row_hash THEN
    UPDATE SET
        t.product_id = s.product_id,
        t.quantity   = s.quantity,
        t.list_price = s.list_price,
        t.discount   = s.discount,
        t.row_hash   = s.row_hash,
        t.updated_at = current_timestamp()

WHEN NOT MATCHED THEN
    INSERT (
        order_id, item_id, product_id,
        quantity, list_price, discount,
        row_hash, updated_at
    ) VALUES (
        s.order_id, s.item_id, s.product_id,
        s.quantity, s.list_price, s.discount,
        s.row_hash, current_timestamp()
    );


In [0]:
%sql
DELETE FROM silver.fact_orders_items t
WHERE NOT EXISTS (
    SELECT 1
    FROM bronze.order_items b
    WHERE t.order_id = b.order_id AND t.item_id = b.item_id
);

In [0]:
%sql

CREATE TABLE IF NOT EXISTS silver.fact_stock_moviments (
    store_id   INT,       -- FK para dim_stores
    product_id INT,       -- FK para dim_product
    quantity   INT,       -- Métrica: quantidade em estoque
    row_hash   STRING,
    updated_at TIMESTAMP
);

WITH staged AS (
    SELECT
        *,
        SHA2(
            CONCAT_WS('|', store_id, product_id, quantity),
            256
        ) AS row_hash
    FROM bronze.stocks
)

MERGE INTO silver.fact_stock_moviments t
USING staged s
ON t.store_id = s.store_id AND t.product_id = s.product_id   -- Chave composta

WHEN MATCHED AND t.row_hash <> s.row_hash THEN
    UPDATE SET
        t.quantity   = s.quantity,
        t.row_hash   = s.row_hash,
        t.updated_at = current_timestamp()

WHEN NOT MATCHED THEN
    INSERT (
        store_id, product_id, quantity,
        row_hash, updated_at
    )
    VALUES (
        s.store_id, s.product_id, s.quantity,
        s.row_hash, current_timestamp()
    );



In [0]:
%sql

DELETE FROM silver.fact_stock_moviments
WHERE NOT EXISTS (
    SELECT 1
    FROM bronze.stocks
    WHERE silver.fact_stock_moviments.store_id = bronze.stocks.store_id
      AND silver.fact_stock_moviments.product_id = bronze.stocks.product_id
);